In [7]:
import cv2
import numpy as np

def track_orange_ball():
    # Initialize webcam video capture
    cap = cv2.VideoCapture(0)

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return

    # Define the range for the color orange in HSV space
    lower_orange = np.array([15, 120, 120])  # Lower bound for orange
    upper_orange = np.array([30, 255, 255])  # Upper bound for orange

    while True:
        # Capture frame-by-frame from webcam
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame")
            break

        # Resize the frame for faster processing
        frame = cv2.resize(frame, (640, 480))

        # Convert the frame from BGR to HSV color space
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Create a mask for the orange color using the specified HSV range
        mask = cv2.inRange(hsv, lower_orange, upper_orange)

        # Apply morphological transformations to remove noise from the mask
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # Find contours of the detected object
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            # Calculate the area of the contour
            area = cv2.contourArea(contour)

            # Ignore small contours to avoid noise
            if area > 500:
                # Approximate the contour to a circle
                (x, y), radius = cv2.minEnclosingCircle(contour)
                center = (int(x), int(y))
                radius = int(radius)

                # Draw the circle around the detected ball
                cv2.circle(frame, center, radius, (0, 255, 0), 3)
                # Optionally, put a label on the ball
                cv2.putText(frame, "Orange Ball", (center[0] - 20, center[1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Show the original frame with tracking results
        cv2.imshow("Orange Ball Tracking", frame)

        # Show the mask (for debugging purposes)
        cv2.imshow("Mask", mask)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture and close all windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    track_orange_ball()